This code replicates PRICE function in MS Excel.
For a detailed explanation of what PRICE function does in MS Excel, refer: https://support.microsoft.com/en-us/office/price-function-3ea9deac-8dfa-436f-a7c8-17ea02c21b0a

Purpose: To compute the PV of of bond with 100 as redemption value and pays periodic coupon.



In [278]:
import datetime
import numpy as np
import pandas as pd
import calendar
import math

In [277]:
def Coupon_Dates(Settlement,Maturity,N):
    if N>1:
               
        SettMonth=Settlement.month
        SettDate=Settlement.day
        SettYear=Settlement.year
    
        MatMonth=Maturity.month
    
        CouponMonth=MatMonth-6
        if CouponMonth<0:
            CouponMonth=CouponMonth+12
        
        CouponDay = 31   
        if (CouponMonth == 4) | (CouponMonth == 6) | (CouponMonth == 9) | (CouponMonth == 11):
            CouponDay = 30
        elif CouponMonth == 2:
            CouponDay = 28
            
        NextCouponDate = datetime.date(SettYear, CouponMonth, CouponDay)
    
        PrevCouponMonth=CouponMonth-6
        PrevCouponYear=SettYear
        if PrevCouponMonth<=0:
            PrevCouponMonth=PrevCouponMonth+12
            PrevCouponYear=SettYear-1
    
        PrevCouponDay = 31   
        if (PrevCouponMonth == 4) | (PrevCouponMonth == 6) | (PrevCouponMonth == 9) | (PrevCouponMonth == 11):
            PrevCouponDay = 30
        elif PrevCouponMonth == 2:
            PrevCouponDay = 28
        
        PrevCouponDate = datetime.date(PrevCouponYear, PrevCouponMonth, PrevCouponDay)
        
    if N==1:
        SettMonth=Settlement.month
        SettDate=Settlement.day
        SettYear=Settlement.year
    
        MatMonth=Maturity.month
    
        PrevCouponMonth=MatMonth-6
        if PrevCouponMonth<0:
            PrevCouponMonth=PrevCouponMonth+12
        
        PrevCouponDay = 31   
        if (PrevCouponMonth == 4) | (PrevCouponMonth == 6) | (PrevCouponMonth == 9) | (PrevCouponMonth == 11):
            PrevCouponDay = 30
        elif PrevCouponMonth == 2:
            PrevCouponDay = 28
            
        PrevCouponDate = datetime.date(SettYear, PrevCouponMonth, PrevCouponDay)
        
    
    return(Maturity,PrevCouponDate)

In [276]:
def fn_Price(Settlement,Maturity,Rate,Yield,Redemption,Frequency,Basis):
      
    Months = (Maturity.year*12 + Maturity.month)-(Settlement.year*12 + Settlement.month)
    DaysLeftInMonth = calendar.monthrange(Settlement.year, Settlement.month)[1] - Settlement.day
    if DaysLeftInMonth>0:
        Months = Months + 1
    
    N = math.ceil(Months * Frequency / 12)
        
    if N>1:
        CouponDates = Coupon_Dates(Settlement,Maturity,N)
        
        DSC= (CouponDates[0]-Settlement).days #number of days from settlement to next coupon date
        A= (Settlement-CouponDates[1]).days #number of days from beginning of coupon period to settlement date.
        E=(CouponDates[0] - CouponDates[1]).days #number of days in coupon period in which the settlement date falls.
               
        ReedemPV = Redemption/(1+Yield/Frequency)**(N-1+(DSC/E)) #Present Value of Redemption Amount
        AccruedInt = 100 * (Rate/Frequency) * (A/E) #Accrued Interest
        CouponPV = 0 #Present Value of Coupon Payment
        for i in range (N):
            CouponPV = CouponPV + ((100 * Rate/Frequency)/(1 + Yield/Frequency)**(i + DSC/E))
                    
        Price = ReedemPV + CouponPV - AccruedInt
        
    if N==1:
        CouponDates = Coupon_Dates(Settlement,Maturity,N)
        
        A= (Settlement-CouponDates[1]).days #number of days from beginning of coupon period to settlement date.
        E=(CouponDates[0] - CouponDates[1]).days #number of days in coupon period in which the settlement date falls.
        DSR=E-A 
        
        T1 = (100*Rate/Frequency) + Redemption
        T2 = ((Yield*DSR)/(Frequency*E))+1
        T3 = 100*Rate*A/(Frequency*E)
        
        Price = (T1/T2)-T3
    
   
    return(Price)
      

In [279]:
Settlement = datetime.date(2024, 7, 1)
Maturity = datetime.date(2024, 12, 31)
Rate = 0.08
Yield = 0.07
Redemption = 100
Frequency = 2 #Assume only 2 is allowed
Basis = 1

Price = fn_Price(Settlement,Maturity,Rate,Yield,Redemption,Frequency,Basis)
print(Price)

100.47982332853425
